# LAB ASSIGNMENT 3
## TEXT ANALYTICS
### NAME:
- AIMI AFIFAH BINTI MUHAMMAD HUSNI (SW01083127)
- SARAH HANI BINTI MOHD AZRAL (SW01083158)

In [1]:
# Import necessary libraries
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from gensim import corpora
from gensim.models import LdaModel

In [2]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...


True

- This code imports necessary libraries: pandas, re, nltk, gensim. It dDownloads NLTK resources for tokenization, stopwords, and WordNet It also p
Provides essential functionalities for text preprocessing and topic modelinta.on.

In [4]:
# Read the data
news_data = pd.read_csv("news_dataset.csv")
news_data.head()

,Unnamed: 0,text,target,title,date
0,0,I was wondering if anyone out there could enli...,7,rec.autos,2022-08-02 13:48:37.251043
1,17,I recently posted an article asking what kind ...,7,rec.autos,2022-08-02 13:48:37.251043
2,29,\nIt depends on your priorities. A lot of peo...,7,rec.autos,2022-08-02 13:48:37.251043
3,56,an excellent automatic can be found in the sub...,7,rec.autos,2022-08-02 13:48:37.251043
4,64,: Ford and his automobile. I need information...,7,rec.autos,2022-08-02 13:48:37.251043


In [5]:
# Drop null values and duplicates
news_data = news_data.dropna(subset=['text'])
news_data = news_data.drop_duplicates()

In [6]:
# Check the shape and the first few rows of the data
print(news_data.shape)
print(news_data.head(20))

(11096, 5)
    Unnamed: 0                                               text  target  \
0            0  I was wondering if anyone out there could enli...       7   
1           17  I recently posted an article asking what kind ...       7   
2           29  \nIt depends on your priorities.  A lot of peo...       7   
3           56  an excellent automatic can be found in the sub...       7   
4           64  : Ford and his automobile.  I need information...       7   
5           71  \nYo! Watch the attributions--I didn't say tha...       7   
6           73  \nYou can avoid these problems entirely by ins...       7   
7           77  I have a 1986 Acura Integra 5 speed with 95,00...       7   
8           84  \nassuming yours is a non turbo MR2, the gruff...       7   
9          156  \n\nIn addition to restricted mileage, many cl...       7   
10         201  \n\nSeveral years ago GM was having trouble wi...       7   
11         207  \nI had the same problem in my '90 MX-6. Lucki...

- The code reads the dataset from a CSV file named "news_dataset.csv" using pandas' read_csv() function. 
It removes null values and duplicates from the 'text' column of the dataset using the dropna() and drop_duplicates() methods, respectively 
After cleaning the data, it prints the shape of the cleaned dataset to display the number of rows and column .
Additionally, it displays the first 20 rows of the cleaned dataset to examine its structure and content. This helps in understanding the data before proceeding with further analysis or preprocessing.

In [7]:
# Select the 'text' column
text_data = news_data['text']

In [8]:
# Check for duplicated text
text_duplicates = text_data.duplicated()
print(text_duplicates.sum())

103


In [9]:
# Define stopwords and initialize stemmer and lemmatizer
stop_words = stopwords.words('english')
more_stopwords = ['u', 'im', 'c', 'maxaxaxaxaxaxaxaxaxaxaxaxaxaxax']
stop_words.extend(more_stopwords)
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [10]:
# Define preprocessing function
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\b\w*\d\w*\b', '', text)  # Remove words containing numbers
    text = re.sub(r'\S+@\S+', '', text)  # Remove emails
    tokens = word_tokenize(text)  # Tokenize text
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    return tokens

- This code snippet sets up the initial steps for text preprocessing. It selects the 'text' column from the dataset and checks for duplicated text. Then, it defines stopwords, initializes a stemmer and a lemmatizer, and creates a preprocessing function. The function converts text to lowercase, removes punctuation, numbers, emails, and stopwords, and performs lemmatization. These steps lay the foundation for cleaning and preparing the text data for further analysis, such as topic modeling.

In [11]:
# Apply preprocessing
news_data['clean_text'] = news_data['text'].apply(preprocess_text)
print(news_data.head(20))

    Unnamed: 0                                               text  target  \
0            0  I was wondering if anyone out there could enli...       7   
1           17  I recently posted an article asking what kind ...       7   
2           29  \nIt depends on your priorities.  A lot of peo...       7   
3           56  an excellent automatic can be found in the sub...       7   
4           64  : Ford and his automobile.  I need information...       7   
5           71  \nYo! Watch the attributions--I didn't say tha...       7   
6           73  \nYou can avoid these problems entirely by ins...       7   
7           77  I have a 1986 Acura Integra 5 speed with 95,00...       7   
8           84  \nassuming yours is a non turbo MR2, the gruff...       7   
9          156  \n\nIn addition to restricted mileage, many cl...       7   
10         201  \n\nSeveral years ago GM was having trouble wi...       7   
11         207  \nI had the same problem in my '90 MX-6. Lucki...       7   

- This code applies the preprocessing steps to the 'text' column of the dataset. It utilizes the previously defined preprocess_text() function to convert the text into lowercase, remove punctuation, numbers, emails, tokenize the text, remove stopwords, and perform lemmatization. The preprocessed text is then stored in a new column named 'clean_text'. The print(news_data.head(20)) statement displays the first 20 rows of the dataset, including the original text and the corresponding preprocessed text. This allows for verification of the preprocessing steps and ensures that the data is ready for further analysis, such as topic modeling with LDA.





In [12]:
# Create a Gensim Dictionary object from the preprocessed documents
dictionary = corpora.Dictionary(news_data['clean_text'])

In [13]:
# Convert each preprocessed document into a bag-of-words representation using the dictionary
corpus = [dictionary.doc2bow(doc) for doc in news_data['clean_text']]

In [14]:
# Perform LDA using Gensim library
lda_model = LdaModel(corpus, num_topics=4, id2word=dictionary, passes=15)

In [15]:
# Create an empty list to store dominant topic labels for each document
article_labels = []

In [16]:
# Iterate over each processed document
for i, doc in enumerate(news_data['clean_text']):
    # Convert each document to bag-of-words representation
    bow = dictionary.doc2bow(doc)
    # Get list of topic probabilities
    topics = lda_model.get_document_topics(bow)
    # Determine topic with highest probability
    dominant_topic = max(topics, key=lambda x: x[1])[0]
    # Append to the list
    article_labels.append(dominant_topic)

- This code applies Latent Dirichlet Allocation (LDA) using the Gensim library to the preprocessed text data. It first creates a dictionary mapping words to numerical IDs from the preprocessed documents. Then, it converts each document into a bag-of-words representation using this dictionary. Next, LDA is performed with a specified number of topics and passes over the corpus. Finally, it determines the dominant topic for each document and stores the corresponding labels.

In [17]:
# Create dataframe to see them
topic_df = pd.DataFrame({"Text": news_data['text'], "Topic": article_labels})
print("Table with Articles and Topics:")
print(topic_df)

Table with Articles and Topics:
                                                    Text  Topic
0      I was wondering if anyone out there could enli...      0
1      I recently posted an article asking what kind ...      0
2      \nIt depends on your priorities.  A lot of peo...      0
3      an excellent automatic can be found in the sub...      0
4      : Ford and his automobile.  I need information...      0
...                                                  ...    ...
11309  Secrecy in Clipper Chip\n\nThe serial number o...      2
11310  Hi !\n\nI am interested in the source of FEAL ...      2
11311  The actual algorithm is classified, however, t...      2
11312  \n\tThis appears to be generic calling upon th...      0
11313  \nProbably keep quiet and take it, lest they g...      0

[11096 rows x 2 columns]


- This part of the code creates a new DataFrame named topic_df to display the original text along with the assigned topic labels. It constructs the DataFrame by combining the 'text' column from the original dataset with the 'article_labels' list generated during the topic modeling process. Each row in the DataFrame represents a piece of text along with its assigned topic label. Finally, it prints out the DataFrame to visually inspect the articles and their corresponding topics. This allows for a better understanding of how the topics are distributed across the dataset.

In [18]:
# Print top terms for each topic
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}:")
    terms = [term.strip() for term in topic.split("+")]
    for term in terms:
        weight, word = term.split("*")
        print(f"- {word.strip()} (weight: {weight.strip()})")
    print()

Topic 0:
- "would" (weight: 0.008)
- "one" (weight: 0.008)
- "people" (weight: 0.007)
- "dont" (weight: 0.005)
- "think" (weight: 0.005)
- "know" (weight: 0.005)
- "like" (weight: 0.004)
- "time" (weight: 0.004)
- "u" (weight: 0.004)
- "say" (weight: 0.004)

Topic 1:
- "db" (weight: 0.022)
- "space" (weight: 0.006)
- "car" (weight: 0.005)
- "mov" (weight: 0.005)
- "power" (weight: 0.005)
- "bike" (weight: 0.004)
- "period" (weight: 0.003)
- "one" (weight: 0.003)
- "wire" (weight: 0.003)
- "bit" (weight: 0.003)

Topic 2:
- "x" (weight: 0.014)
- "key" (weight: 0.009)
- "use" (weight: 0.007)
- "system" (weight: 0.007)
- "file" (weight: 0.006)
- "chip" (weight: 0.005)
- "one" (weight: 0.005)
- "encryption" (weight: 0.005)
- "get" (weight: 0.004)
- "program" (weight: 0.004)

Topic 3:
- "game" (weight: 0.010)
- "team" (weight: 0.009)
- "year" (weight: 0.005)
- "player" (weight: 0.005)
- "new" (weight: 0.005)
- "university" (weight: 0.004)
- "season" (weight: 0.004)
- "league" (weight: 0.004)

- This section of the code prints the top terms for each topic identified by the LDA model. For each topic, it iterates through the list of terms and their corresponding weights, printing them out in a human-readable format. Each line displays a term along with its weight within the topic. These terms represent the most relevant words associated with each topic as identified by the LDA model. Understanding these top terms can provide insight into the themes or subjects covered by each topic.

In [19]:
# Print top terms for each topic
for topic_id in range(lda_model.num_topics):
    print(f"Top terms for Topic #{topic_id}:")
    top_terms = lda_model.show_topic(topic_id, topn=10)
    print([term[0] for term in top_terms])
    print()

Top terms for Topic #0:
['would', 'one', 'people', 'dont', 'think', 'know', 'like', 'time', 'u', 'say']

Top terms for Topic #1:
['db', 'space', 'car', 'mov', 'power', 'bike', 'period', 'one', 'wire', 'bit']

Top terms for Topic #2:
['x', 'key', 'use', 'system', 'file', 'chip', 'one', 'encryption', 'get', 'program']

Top terms for Topic #3:
['game', 'team', 'year', 'player', 'new', 'university', 'season', 'league', 'play', 'win']



- This code also prints the top terms for each topic identified by the LDA model. It iterates through each topic and utilizes the show_topic() method to retrieve the top N terms for that topic, where N is specified by the topn parameter. It then prints out these top terms for each topic, providing insight into the main themes or concepts associated with each topic. This representation can be helpful in interpreting and understanding the topics generated by the LDA model.